<h3>Cadeias de Markov na Geração de Jogos da Loteria</h3>

<h4>Declaração de Variáveis e Visualização dos Dados</h4>

In [2]:
from Neuraline.Utilities.data import DataTable, DataExploration # importação das classes
data_table, data_exploration = DataTable(), DataExploration() # instanciação dos objetos
# declaração das variáveis de endereçamento e dos títulos das colunas que serão utilizadas na análise
url_path, sheet_name, url_description, url_save = './mega_sena.xlsx', 'mega_sena_www.asloterias.com.br', './description.xlsx', './jogos.csv'
titles = ['bola 1', 'bola 2', 'bola 3', 'bola 4', 'bola 5', 'bola 6']
result = data_table.visualization_excel_inline(url_path, sheet_name=sheet_name, limit=10) # visualização da estrutura do arquivo de dados

╒════════════╤════════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│   Concurso │ Data       │   bola 1 │   bola 2 │   bola 3 │   bola 4 │   bola 5 │   bola 6 │
╞════════════╪════════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│       2515 │ 31/08/2022 │       54 │        3 │       41 │       12 │       45 │       19 │
├────────────┼────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│       2514 │ 27/08/2022 │        5 │       15 │       52 │       34 │       45 │       24 │
├────────────┼────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│       2513 │ 24/08/2022 │       50 │       21 │       13 │       46 │       35 │       19 │
├────────────┼────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│       2512 │ 20/08/2022 │       47 │       52 │       10 │       49 │       34 │        7 │
├────────────┼────────────┼──────────┼──────────┼──────────┼

<h4>Exploração e Análise dos Dados (somente para as 10 primeiras linhas)</h4>

In [3]:
data_exploration.describeExcel(url_path, titles=titles, sheet_name=sheet_name) # cria a tabela de resultados estatísticos
result = data_table.visualization_excel_inline(url_description, sheet_name=sheet_name, limit=10) # visualiza as 10 primeiras linhas do arquivo gerado

╒══════════╤═══════════╤═══════════╤═══════════╤═════════════════════╤══════════════════════╤═════════════╤═════════╤═════════════╤════════╤═════════╕
│ column   │   minimum │   maximum │   average │   lowest_repetition │   highest_repetition │   negatives │   zeros │   positives │   null │   empty │
╞══════════╪═══════════╪═══════════╪═══════════╪═════════════════════╪══════════════════════╪═════════════╪═════════╪═════════════╪════════╪═════════╡
│ bola 1   │         1 │        60 │   30.5237 │                  15 │                   28 │           0 │       0 │        2515 │      0 │       0 │
├──────────┼───────────┼───────────┼───────────┼─────────────────────┼──────────────────────┼─────────────┼─────────┼─────────────┼────────┼─────────┤
│ bola 2   │         1 │        60 │   30.107  │                  60 │                    5 │           0 │       0 │        2515 │      0 │       0 │
├──────────┼───────────┼───────────┼───────────┼─────────────────────┼──────────────────────┼─

<h4>Treinamento e Predição do Modelo com Geração do Arquivo CSV</h4>

In [6]:
from Neuraline.ArtificialIntelligence.MachineLearning.AutonomousLearning.monte_carlo import MonteCarlo # importação da classe de Markov
monte_carlo = MonteCarlo(progress=True) # progress=True: habilita a exibição do progresso do treinamento e da predição

inputs = data_table.read_excel(url_path, titles=titles, sheet_name=sheet_name, limit=365) # leitura dos resultados históricos do ano
monte_carlo.fit(inputs=inputs) # treinamento do modelo

outputs = monte_carlo.predict(predictions=10)
# código para eliminar números repetidos do sorteio
maximum_column = data_table.read_excel(url_description, titles=['maximum'], sheet_name=sheet_name) # captura os valores da coluna maximum
maximum = max([x[0] for x in maximum_column]) # percorre a lista bidimensional para convertê-la em unidimensional e extrai o maior valor da lista
middle = maximum/2 # calcula a metade do limite numérico
temp_outputs = [] # lista de saídas temporárias
for output in outputs: # percorre a lista de resultados preditivos
    temp_output = [] # saída temporária
    for number in output: # percorre o resultado preditivo
        if number not in temp_output: temp_output.append(number) # se o número não constar na lista de saída temporária então será inserido
        else: # se o número já existir na lista de saída temporária uma nova avaliação é feita
            # condicional para evitar que os novos números não excedam os limites mínimo e máximo do sorteio
            if number < middle: number+=1 # se o número for menor que a metade então soma 1
            else: number-=1 # se o número for maior ou igual a metade então subtrai 1
            if number not in temp_output: temp_output.append(number) # se o novo número não constar na lista de saída temporária então será inserido
            else: # se o novo número já existir na lista de saída temporária uma nova avaliação será feita
                for n in range(1, maximum+1): # loop de 1 até o número máximo permitido pelo sorteio
                    if n not in temp_output: # verifica se um dos números possíveis não consta na lista de saída temporária
                        temp_output.append(n) # ao encontrar um número que não conste na lista de saída temporária ele será inserido
                        break # encerra o loop quando o primeiro número inédito for encontrado
    temp_outputs.append(sorted(temp_output)) # insere a saída temporária na lista de saídas temporárias
outputs = temp_outputs # atualiza a lista de resultados preditivos com os valores ordenados sem repetição  

result = data_table.generation_csv(url_save, titles=titles, data=outputs) # geração do arquivo de resultados

training progress............................: 001.000000%
training progress............................: 010.000000%
training progress............................: 020.000000%
training progress............................: 050.000000%
training progress............................: 090.000000%
training progress............................: 100.000000%
prediction progress..........................: 001.000000%
prediction progress..........................: 050.000000%
prediction progress..........................: 055.000000%
prediction progress..........................: 075.000000%
prediction progress..........................: 090.000000%
prediction progress..........................: 100.000000%


<h4>Leitura e Visualização do Arquivo de Resultados</h4>

In [7]:
result = data_table.visualization_csv_inline(url_save) # visualização do arquivo de resultados

╒══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│   bola 1 │   bola 2 │   bola 3 │   bola 4 │   bola 5 │   bola 6 │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│       11 │       27 │       36 │       40 │       58 │       59 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│       21 │       27 │       41 │       42 │       59 │       60 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        2 │        6 │       21 │       37 │       47 │       50 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        1 │       21 │       38 │       41 │       56 │       57 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│       15 │       29 │       40 │       42 │       46 │       56 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        5 │       14 │       35 │       38 │       44 │       60 │
├──────────┼──────────┼──────────┼──────────┼───